<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/325/woman-detective-medium-light-skin-tone_1f575-1f3fc-200d-2640-fe0f.png" width="120">

#####<center>This notebook is based on this amazing ISAE [course](https://supaerodatascience.github.io/OBD/).

## Install dependencies

In [ ]:
!rm -rf /content/data-no-blabla-streamlit
!git clone https://github.com/galleon/data-no-blabla-streamlit.git
!pip install -r data-no-blabla-streamlit/yolo/backend/requirements.txt
!pip install flask-cloudflared
# localtunnel is down since 24/06
# !npm install -g localtunnel

# Run the backend web server

In [ ]:
%cd data-no-blabla-streamlit/yolo/backend/

In [ ]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port,))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  !uvicorn app:app --port {app_port}

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    t1.join()

### Now, take the url you got from localtunnel and use it in the client notebook. 

### Let the fun begin ✨